In [1]:
import torch


In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-05 22:02:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.008s  

2025-03-05 22:02:49 (131 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [52]:
batch_size = 16 # how many independent sequences will we process in parallel?
sequence_length = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
embed_dim = 64
num_head = 4
num_layers = 4

In [53]:
with open('/content/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [54]:
chars = sorted(list(set(text)))
print("".join([i for i in chars]))
len(chars)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [55]:
char_to_index = {ch:i for i,ch in enumerate(chars)}
index_to_char = {i:ch for i,ch in enumerate(chars)}

In [56]:
char_to_index
encode = lambda s: [ char_to_index[i] for i in s]
decode = lambda s: "".join([ index_to_char[i] for i in s])

In [57]:
data = torch.tensor(encode(text))

In [58]:
train_len = 0.9*(len(data))

In [59]:
training_data = data[:int(train_len)]
validation_data = data[int(train_len):]

In [60]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

def get_bacthes(split):
  data = training_data if split == 'train' else validation_data
  idx = torch.randint(len(data)-sequence_length,(batch_size,))
  X = torch.stack([data[i:sequence_length+i] for i in idx])
  Y = torch.stack([data[i+1:sequence_length+1+i] for i in idx])
  return X,Y


X,Y = get_bacthes('train')
X

tensor([[58, 53,  1, 41, 53, 56, 56, 59, 54, 58,  1, 39,  1, 51, 39, 52,  5, 57,
          1, 61, 47, 44, 43,  1, 47, 57,  0, 61, 46, 43, 52,  1],
        [49,  1, 39, 52,  1, 53, 39, 58, 46,  1, 40, 63,  1, 20, 47, 51,  6,  0,
         32, 46, 43,  1, 59, 52, 47, 58, 63,  1, 58, 46, 43,  1],
        [59, 50, 42,  1, 58, 46, 53, 59,  1, 61, 43, 56, 58,  1, 57, 53,  1, 58,
         53, 53,  2,  0,  0, 24, 33, 15, 21, 27, 10,  0, 35, 43],
        [ 8,  0,  0, 35, 13, 30, 35, 21, 15, 23, 10,  0, 28, 56, 53, 60, 43,  1,
         47, 58,  6,  1, 20, 43, 52, 56, 63,  6,  1, 39, 52, 42],
        [58,  1, 57, 46, 43,  8,  0,  0, 32, 30, 13, 26, 21, 27, 10,  0, 18, 53,
         56,  1, 61, 46, 39, 58,  1, 56, 43, 39, 57, 53, 52,  6],
        [56, 61, 47, 41, 49,  6,  1, 50, 43, 58,  1, 47, 58,  1, 40, 43, 11,  0,
         18, 53, 56,  1, 47, 52,  1, 58, 46, 63,  1, 57, 46, 53],
        [25, 10,  0, 35, 47, 58, 46, 42, 56, 39, 61,  1, 63, 53, 59,  1, 46, 43,
         52, 41, 43,  6,  1, 51, 63, 

In [61]:
class SelfAttentionHead(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.query = nn.Linear(embed_dim,head_size,bias=False)
    self.key = nn.Linear(embed_dim,head_size,bias=False)
    self.value = nn.Linear(embed_dim,head_size,bias=False)
    self.register_buffer('trill',torch.tril(torch.ones(sequence_length,sequence_length)))
    self.dropout = nn.Dropout(0.2)

  def forward(self,x):
    B,T,C = x.shape
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    weightedIndex = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
    masked_wei = weightedIndex.masked_fill(self.trill[:T,:T]==0,float('-inf'))
    sofmaxwei = F.softmax(masked_wei,dim=-1)
    out = self.dropout(sofmaxwei)
    out = out @ v
    return out



In [62]:
class MultiHeadAttention(nn.Module):
  def __init__(self,num_head,head_size):
    super().__init__()
    self.heads = nn.ModuleList([SelfAttentionHead(head_size=head_size) for i in range(num_head)])
    self.dropout = nn.Dropout(0.2)
    self.projection = nn.Linear(num_head * head_size,embed_dim)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim=-1)
    out = self.dropout(out)
    return out

In [63]:
class Block(nn.Module):
  def __init__(self,embed_dim,num_head):
    super().__init__()
    head_size = embed_dim // num_head
    self.multihead = MultiHeadAttention(num_head,head_size)
    self.ff = FeedForward(embed_dim)
    self.ln1 = nn.LayerNorm(embed_dim)
    self.ln2 = nn.LayerNorm(embed_dim)


  def forward(self,x):
    x = x + self.multihead(self.ln1(x))
    x = x + self.ff(self.ln2(x))
    return x


In [64]:
class FeedForward(nn.Module):
  def __init__(self,embed_dim):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(embed_dim,4*embed_dim),
        nn.ReLU(),
        nn.Linear(4*embed_dim,embed_dim),
        nn.Dropout(0.2)
    )

  def forward(self,x):
    out = self.layers(x)
    return out


In [65]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)



device = torch.device("cpu")
class BigramModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding_table = nn.Embedding(vocab_size,embed_dim)
    self.position_embedding_table  = nn.Embedding(sequence_length,embed_dim)
    self.projection = nn.Linear(embed_dim,vocab_size)
    self.blocks = nn.Sequential(*[Block(embed_dim=embed_dim,num_head=num_head)for i in range(num_layers)])
    self.layernorm = nn.LayerNorm(embed_dim)

  def forward(self,idx,targets=None):
    B,T = idx.shape
    embeddings = self.embedding_table(idx)
    positions = torch.arange(T,device=device)
    position_embeddigs = self.position_embedding_table(positions)
    logits = embeddings + position_embeddigs
    logits = self.blocks(logits)
    logits = self.layernorm(logits)
    logits = self.projection(logits)

    if targets is None:
      loss = None

    else:
      B,T,C = logits.shape
      logits = logits.reshape(B*T,C)
      targets = targets.reshape(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx,max_new_tokens):
    for i in range(max_new_tokens):
      idx_cond = idx[:, -sequence_length:]
      logits,loss = self(idx_cond)
      current_tokens = logits[:,-1,:]
      probs = F.softmax(current_tokens,dim=-1)
      indx = torch.multinomial(probs,num_samples=1).long()
      idx = torch.cat((idx,indx),dim=-1)
    return idx

m = BigramModel(len(chars)).to(device)
idx = torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(m.generate(idx,max_new_tokens=500)[0].tolist()))


AnA&qUUBAy;xGLhVt.jAXio!w3UCj&cUNwVvoB!EIJkVBl.bj
!Vr!JraeZ?eKAFImwESq& hoRH'w!F W,Cz.&GY!CA,XcM'XL3,iWSJIpXkvH&I!m3MLfpjJivqxAA.;S',TkVgnTgre,pz'x iFT&TWy'O',CaFIqRD!pwu?.kSTBnSGqVRcU& LRMgAcnv-vemdZRjYDg;';::QADQV'u n$$&'A;J;V-XxSXRg$
-ocRsXU,rJi hTXO,h3?,3XLp&rP tD3z;o!mRzeXtT-,ep
,anCV&m
 oeSw!i;Vd';ApXxLf!$GHhW
X;OQ&!O,G?$$
&VrGn;x3IRTXsb $C&ISs&SiFtk;ocmXvV'Jrz:DP-X'nkbjC,T&Xl-AOf-SS-eAASECKVk
Dz!Juqsr,hxm;xjJOtJhNjGxkN$dR&wk!q?R!KXR,g&OTH!YAFLwq
ijKRZ: N!j;'A-&Rrp
pHWQNAyoOPYMXjD3,PV&rS$T


In [66]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_bacthes(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



model = BigramModel(vocab_size=len(chars))
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_bacthes('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.3738, val loss 4.3592
step 100: train loss 2.6784, val loss 2.6759
step 200: train loss 2.5357, val loss 2.5377
step 300: train loss 2.4713, val loss 2.4688
step 400: train loss 2.4093, val loss 2.4174
step 500: train loss 2.3533, val loss 2.3461
step 600: train loss 2.3097, val loss 2.3103
step 700: train loss 2.2659, val loss 2.2751
step 800: train loss 2.2424, val loss 2.2457
step 900: train loss 2.2070, val loss 2.2268
step 1000: train loss 2.1862, val loss 2.1966
step 1100: train loss 2.1473, val loss 2.1687
step 1200: train loss 2.1226, val loss 2.1440
step 1300: train loss 2.1087, val loss 2.1350
step 1400: train loss 2.0862, val loss 2.1241
step 1500: train loss 2.0791, val loss 2.1150
step 1600: train loss 2.0568, val loss 2.1089
step 1700: train loss 2.0385, val loss 2.0860
step 1800: train loss 2.0311, val loss 2.0701
step 1900: train loss 2.0255, val loss 2.0616
step 2000: train loss 2.0016, val loss 2.0510
step 2100: train loss 1.